In [1]:
import urllib.request, json

with urllib.request.urlopen(
    "https://api.steinhq.com/v1/storages/5e736c1db88d3d04ae0815b3/Raw_Data"
) as url:
    data = json.loads(url.read().decode())
    

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(data)

In [4]:
df["Notes"][30:35]

30              Travelled to Thailand and Malaysia
31                             Travelled from Iran
32                             Travelled from Iran
33                             Travelled from Oman
34    Travel from Italy on 29/02/2020 through Doha
Name: Notes, dtype: object

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [16]:
# import spacy
# from spacy import displacy
# from spacy.matcher import Matcher

# nlp = spacy.load("en_core_web_sm")
# matcher = Matcher(nlp.vocab)
# matched_sents = []  # Collect data of matched sentences to be visualized

In [17]:
# pattern = [{"LEMMA": "travel"}, {"LOWER": "from"}, {"POS": "PROPN"}]
# matcher.add("TravelledFrom", None, pattern)
# doc = nlp("Travelled from Italy and China")
# matches = matcher(doc)
# matches

In [18]:
# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(span.text)

In [19]:
# from spacy.tokens import Token
# def find_travelled_from(sents):
#     matcher = Matcher(nlp.vocab)
#     pattern = [{"LEMMA": "travel"}, {"LOWER": "from"}]
#     matcher.add("TravelledFrom", None, pattern)
#     Token.set_extension("is_travelled_from", default=False, force=True)
#     for sent in sents:
#         doc = nlp(sent)
#         matches = matcher(doc)
#         travelled_from = []
#         for match_id, start, end in matches:
#             if doc.vocab.strings[match_id] == "TravelledFrom":
#                 travelled_from.append(doc[start:end])
#         with doc.retokenize() as retokenizer:
#             for span in travelled_from:
#                 retokenizer.merge(span)
#                 for token in span:
#                     token._.is_travelled_from = True
#         for token in doc:
#             if token._.is_travelled_from:
#                 print(token.text, token._.is_travelled_from)
        

In [20]:
# input_sents = list(df["Notes"][30:35])
# find_travelled_from(input_sents)

In [21]:
# df["Notes"][35]

In [22]:
# def find_travelled_from(sents):
#     matcher = Matcher(nlp.vocab)
#     pattern = [{"LEMMA": "travel"}, {}]
#     matcher.add("Travel", None, pattern)
#     for sent in sents:
#         doc = nlp(sent)
#         matches = matcher(doc)
#         for match_id, start, end in matches:
#             string_id = nlp.vocab.strings[match_id]  # Get string representation
#             span = doc[start:end]  # The matched span
#             print("Input: ", sent, "\nPlace: ", span.text.split()[-1])

In [23]:
from spacy.tokens import Span

def get_travel_status(span):
    if span.label_ =="GPE":
        prev_token = span.doc[span.start - 1]
        if prev_token.text in ("from", "through", "via", "Via"):
            return("from")
        elif prev_token.text in ("to", "and"):
            return("to")
        return "to"

# Register the Span extension as 'travel_status'
Span.set_extension("travel_status", getter=get_travel_status, force=True)

In [24]:
from spacy.tokens import Span

def get_nationality(span):
    if span.label_ =="NORP":
        return span.text

# Register the Span extension as 'nationality'
Span.set_extension("nationality", getter=get_nationality, force=True)

doc = nlp("Indian Tourist")
print([(ent.text, ent._.nationality) for ent in doc.ents])

[('Indian', 'Indian')]


In [32]:
from spacy.matcher import Matcher

In [140]:
# from spacy.tokens import Token
# def find_family_member(sent):
#     matcher = Matcher(nlp.vocab)
#     pattern = [{"LOWER": "family"}, {"LOWER": "member"}, {"LOWER": "of"}, {}]
#     matcher.add("FamilyMember", None, pattern)
#     Token.set_extension("is_family_member", default=False, force=True)
#     doc = nlp(sent)
#     matches = matcher(doc)
#     travelled_from = []
#     for match_id, start, end in matches:
#         if doc.vocab.strings[match_id] == "FamilyMember":
#             travelled_from.append(doc[start:end])
#     with doc.retokenize() as retokenizer:
#         for span in travelled_from:
#             retokenizer.merge(span)
#             for token in span:
#                 token._.is_family_member = True
#     return doc
# [tok._.is_family_member for tok in find_family_member("Family Member of group 13")]

[True, False]

In [207]:
from spacy.tokens import Span


def get_relationship(token):
    if token.text == "of":
        prev_token = token.doc[token.i - 1]
        prev2 = None
        if token.i > 2:
            prev2 = token.doc[token.i - 2]
            if prev2.text.lower() == "and":
                return f"{token.doc[token.i - 3]} {token.doc[token.i - 2]} {token.doc[token.i - 1]}"
        if prev_token.text.lower() in ("members", "member"):
            return "Family Member"
        else:
            return prev_token.text


# Register the Span extension as 'relationship'
Token.set_extension("relationship", getter=get_relationship, force=True)

doc = nlp("friend and family of p23")
print([(ent.text, ent._.relationship) for ent in doc])

[('friend', None), ('and', None), ('family', None), ('of', 'friend and family'), ('p23', None)]


In [216]:
import re
def find_travelled(sents):
    for sent in sents:
        if not sent:
            return
        s = re.sub(r'[^\w\s]',' ',sent)
        doc = nlp(s)
        print(sent)
        print("---")
        for tok in doc:
            if tok._.relationship:
                print(f"\tRelationship : {tok._.relationship}")
        for ent in doc.ents:
            if ent._.travel_status:
                print(f"\tTravel Place : {ent.text}")
            if ent._.nationality:
                print(f"\tNationality : {ent._.nationality}")
        print("---")
            

In [217]:
find_travelled(df["Notes"])

Student from Wuhan, recovery confirmed on 14 Feb
---
	Travel Place : Wuhan
---
Student from Wuhan, recovery confirmed on 14 Feb
---
	Travel Place : Wuhan
---
Student from Wuhan, recovery confirmed on 14 Feb
---
	Travel Place : Wuhan
---
Travel history to Italy and Austria
---
	Travel Place : Italy
	Travel Place : Austria
---
Travel history to Dubai, Singapore contact
---
	Travel Place : Dubai
	Travel Place : Singapore
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian tourist
---
	Nationality : Italian
---
Italian to

Returned from UAE
---
---
Philippines nationals who had come to Navi Mumbai
---
	Travel Place : Philippines
	Travel Place : Mumbai
---
Philippines nationals who had come to Navi Mumbai
---
	Travel Place : Philippines
	Travel Place : Mumbai
---
Spouse of P95, From Kalyan
---
	Relationship : Spouse
---
3 year old from Kalyan, Father P95 /Mother P122
---
---
Travel history to Lisbon in Portugal
---
	Travel Place : Lisbon
	Travel Place : Portugal
---
Patient returned from Dubai on March 1
---
	Travel Place : Dubai
---
Returned from Italy
---
	Travel Place : Italy
---
Returned from Scotland
---
	Travel Place : Scotland
---
2nd evacuee from Italy in ITBP Chhawla Camp
---
	Travel Place : Italy
---
3rd evacuee from Italy in ITBP Chhawla Camp
---
	Travel Place : Italy
---
Travelled to Singapore (Feb) and Canada (Mar)
---
	Travel Place : Singapore
	Nationality : Feb
	Travel Place : Canada
---
Sector 9; travelled from Malaysia
---
	Travel Place : Malaysia
---
Ladakh
---
---
Ladakh
---
---
Ladakh


In [233]:
def get_relationship(sent):
    if not sent:
            return
    s = re.sub(r'[^\w\s]',' ',sent)
    doc = nlp(s)
    for tok in doc:
        if tok._.relationship:
            return(tok._.relationship)

def get_travel_place(sent):
    if not sent:
            return
    s = re.sub(r'[^\w\s]',' ',sent)
    doc = nlp(s)
    travel = []
    for ent in doc.ents:
        if ent._.travel_status:
            travel.append(ent.text)
    return travel

def get_nationality(sent):
    if not sent:
            return
    s = re.sub(r'[^\w\s]',' ',sent)
    doc = nlp(s)
    nat = []
    for ent in doc.ents:
        if ent._.nationality:
            nat.append(ent._.nationality)
    return nat

In [234]:
import re
def find_travelled(data):
    df = data.copy()
    df["Relationship"] = df["Notes"].apply(lambda x: get_relationship(x))
    df["Travel Place"] = df["Notes"].apply(lambda x: get_travel_place(x))
    df["Nationality"] = df["Notes"].apply(lambda x: get_nationality(x))
    return df
    

In [236]:
find_travelled(df).to_csv("relationships.csv")

In [228]:
df.iloc[23]

Patient number                                                                              24
Date Announced                                                                      04/03/2020
Estimated Onset Date                                                                      None
Age Bracket                                                                               None
Gender                                                                                    None
Detected City                                                                             Agra
Detected District                                                                         Agra
Detected State                                                                   Uttar Pradesh
Current Status                                                                    Hospitalized
Notes                                                                     Family members of P4
Contracted from which Patient (Suspected)         